In [12]:
import cobra
from cobra.io import read_sbml_model,load_json_model,write_sbml_model
from cobra import Model, Reaction, Metabolite
from cobra.medium import minimal_medium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from rpcfba import *

In [13]:
model = load_json_model('../data/GSMM_parameters/LP_WCFS1.json')
model.objective = {model.reactions.biomass_LPL60: 1}

In [14]:
# write_sbml_model(model, "../data/GSMM_parameters/LP_WCFS1.xml")

# GPR

In [ ]:
# gpr_data=[]
# for gene in  model.genes:
#     for rxn in list(gene.reactions):
#         gpr_data.append({'gene':gene.id,'rxn':rxn.id})    
# gpr_pd = pd.DataFrame(gpr_data)   
# gpr_pd.to_csv('../data/GSMM_parameters/gpr_metnet.csv',index=None)

In [ ]:
# gpr_pd = pd.read_csv('../data/GSMM_parameters/gpr_metnet.csv')
# uniprot_info = pd.read_csv('../data/Proteomics/uniprotkb_taxonomy_id_220668.tsv',sep='\t')
# uniprot_info = uniprot_info.rename({'Gene Names (ordered locus)':'locus_tag'}, axis='columns')
# metgene_tags = list( set(list(gpr_pd['gene'])) )
# print(len(metgene_tags))
# matched_pd = (uniprot_info[uniprot_info.locus_tag.isin(metgene_tags)]).reset_index().drop(['index'],axis=1)
# metgene_uids = list(matched_pd.Entry)+['F9UST1']

In [ ]:
# wcfs1_proteins = SeqIO.parse( "../data/Proteomics/wcfs1_prot.fasta", "fasta")
# met_proteins = []
# for seq_record in wcfs1_proteins:
#     if str(seq_record.id) in metgene_uids:
#         if str(seq_record.id) == 'F9UST1':
#             locus_tag = 'lp_0106/lp_0107'
#         else:
#             locus_tag = list(matched_pd[matched_pd['Entry'] == str(seq_record.id)]['locus_tag'])[0]
#         met_proteins.append( SeqRecord(Seq( str(seq_record.seq).replace('*','') ),\
#                                     id = locus_tag+'|'+str(seq_record.id), name="",description="") )
# SeqIO.write( met_proteins, '../data/Genome_HMX2/wcfs1_metabolic_proteins.fasta' ,"fasta")

# Remove genes

In [3]:
model.remove_reactions([model.reactions.GLCt2r, model.reactions.PKL,model.reactions.GALM,
                model.reactions.F6PA, model.reactions.DHAPT],  remove_orphans=True)
model.remove_metabolites([model.metabolites.a_gal__D_c])

# Add reactions

In [4]:
model.reactions.GALK2.id = 'GALK'
model.reactions.GALK.build_reaction_from_string('atp_c + gal_c --> adp_c + gal1p_c + h_c')
model.reactions.FBA.id = 'FDPA'#avoid confusion with FBA(flux balance analysis)
#fix directions
model.reactions.FDPA.lower_bound = 0.0#FBA
model.reactions.GAPD.lower_bound = 0.0
model.reactions.PGK.lower_bound = 0.0
model.reactions.PGM.lower_bound = 0.0
model.reactions.ENO.lower_bound = 0.0
model.reactions.LDH_L.build_reaction_from_string('h_c + nadh_c + pyr_c --> lac__L_c + nad_c')
model.reactions.LDH_L.lower_bound = 0.0
model.reactions.LDH_D.build_reaction_from_string('h_c + nadh_c + pyr_c --> lac__D_c + nad_c')
model.reactions.LDH_D.lower_bound = 0.0
model.reactions.PFL.lower_bound = 0.0
model.reactions.PTAr.lower_bound = 0.0
model.reactions.ACKr.build_reaction_from_string('actp_c + adp_c --> ac_c + atp_c')
model.reactions.ACKr.lower_bound = 0.0
model.reactions.GALU.lower_bound = 0.0 # irreversible
model.reactions.PGMT.build_reaction_from_string('g6p_c --> g1p_c')
model.reactions.PGMT.lower_bound = 0.0
model.reactions.PMANM.build_reaction_from_string('man6p_c --> man1p_c')
model.reactions.PMANM.lower_bound = 0.0
model.reactions.GALT.lower_bound = 0.0
model.reactions.ACALD.build_reaction_from_string('accoa_c + h_c + nadh_c --> acald_c + coa_c + nad_c')
model.reactions.ACALD.lower_bound = 0.0
model.reactions.ALCD2x.build_reaction_from_string('acald_c + h_c + nadh_c --> etoh_c + nad_c')
model.reactions.ALCD2x.lower_bound = 0.0
# remove ATP maintainance
model.reactions.ATPM.lower_bound = 0.0

In [5]:
gal_e = Metabolite('gal_e','C6H12O6','D-Galactose',0,'default')
EX_gal_e = Reaction('EX_gal_e','D-Galactose exchange', -1000, 1000) ;
GALt2 = Reaction('GALt2','D-Galactose exchange', -1000, 1000) ;
# Build forward and reverse reactions
model.reactions.RPI.id = 'RPI_for';model.reactions.RPI_for.lower_bound = 0.0;
RPI_rev = Reaction('RPI_rev','ribose 5 phosphate isomerase(reverse)', 0, 1000)
model.reactions.RPE.id = 'RPE_for';model.reactions.RPE_for.lower_bound = 0.0;
RPE_rev = Reaction('RPE_rev','ribulose 5 phosphate 3 epimerase(reverse)', 0, 1000)
model.reactions.PGI.id = 'PGI_for'; model.reactions.PGI_for.lower_bound = 0.0;
PGI_rev = Reaction('PGI_rev','glucose 6 phosphate isomerase(reverse)', 0, 1000)
model.reactions.UDPG4E.id = 'UDPG4E_for'; model.reactions.UDPG4E_for.lower_bound = 0.0;
UDPG4E_rev = Reaction('UDPG4E_rev','UDPglucose 4 epimerase(reverse)', 0, 1000)
model.reactions.MAN6PI.id = 'MAN6PI_rev'; model.reactions.MAN6PI_rev.lower_bound = 0.0;
MAN6PI_for = Reaction('MAN6PI_for','mannose 6 phosphate isomerase(forward)', 0, 1000)
model.add_reactions([RPI_rev, RPE_rev, PGI_rev, UDPG4E_rev, MAN6PI_for, EX_gal_e, GALt2])
model.add_metabolites([gal_e])


model.reactions.EX_gal_e.build_reaction_from_string('gal_e <=>')
model.reactions.GALt2.build_reaction_from_string('gal_c + h_c <=> gal_e + h_e')
model.reactions.RPI_rev.build_reaction_from_string('ru5p__D_c --> r5p_c')
model.reactions.RPE_rev.build_reaction_from_string('xu5p__D_c --> ru5p__D_c')
model.reactions.PGI_rev.build_reaction_from_string('f6p_c --> g6p_c')
model.reactions.UDPG4E_rev.build_reaction_from_string('udpgal_c --> udpg_c')
model.reactions.MAN6PI_for.build_reaction_from_string('f6p_c --> man6p_c')

In [6]:
# shut down acetate production
model.reactions.ACKr.upper_bound = 0.0

# Pseudo-reactions of EPS repeating unit synthesis

In [7]:
gdpmann_c = Metabolite('gdpmann_c','C16H23N5O16P2','GDP-D-mannose',-2,'default')
manx14_c = Metabolite('manx14_c','C84H154O70','Mannose residues of EPS repeating unit',0,'default')
manx14_glcx6_c = Metabolite('manx14_glcx6_c','C120H220O100','Mannose and glucose residues of EPS repeating unit',0,'default')
EPS_c = Metabolite('EPS_c','C126H231O105','EPS repeating unit',0,'default')
EPS_e = Metabolite('EPS_e','C126H231O105','EPS repeating unit',0,'default')
model.add_metabolites([gdpmann_c, manx14_c,manx14_glcx6_c,EPS_c, EPS_e ])

MAN1PT = Reaction('MAN1PT','Mannose-1-phosphate guanylyltransferase (GDP)', 0, 1000)
MANT_EPS = Reaction('MANT_EPS','Mannosyltransferase for EPS formation', 0, 1000)
GLCT_EPS = Reaction('GLCT_EPS','Glucosyltransferase for EPS formation', 0, 1000)
GALT_EPS = Reaction('GALT_EPS','Galactosyltransferase for EPS formation', 0, 1000)
WZX = Reaction('WZX','Wzx flippase-mediated membrane translocation','',0,1000)
EX_EPS_e = Reaction('EX_EPS_e','EX_EPS_e','',-1000,1000)
model.add_reactions([MAN1PT, MANT_EPS, GLCT_EPS,GALT_EPS, WZX,EX_EPS_e ])

model.reactions.MAN1PT.build_reaction_from_string('gdp_c + h_c + man1p_c --> gdpmann_c + pi_c')
model.reactions.MANT_EPS.build_reaction_from_string('gdpmann_c --> 0.07143 manx14_c + gdp_c')
model.reactions.GLCT_EPS.build_reaction_from_string('udpg_c + 0.16667 manx14_c --> 0.16667 manx14_glcx6_c + udp_c')
model.reactions.GALT_EPS.build_reaction_from_string('udpgal_c + manx14_glcx6_c --> EPS_c + udp_c')
model.reactions.WZX.build_reaction_from_string('EPS_c + atp_c + h2o_c --> EPS_e + adp_c + pi_c +h_c' )
model.reactions.EX_EPS_e.build_reaction_from_string('EPS_e <=> ')

# Save the modified model

In [8]:
# cobra.io.save_json_model(model, "../data/GSMM_parameters/LP_HMX2.json")

# Assess amino acid requirement

In [5]:
model = load_json_model('../data/GSMM_parameters/LP_HMX2.json')

In [6]:
with model:
    model.medium = init_MRSmedium()
    minmed = minimal_medium(model,0.6, minimize_components=True)
print(minmed)

EX_btn_e       6.000000e-06
EX_cys_L_e     4.825200e-02
EX_glc_e       2.344413e+01
EX_gln_L_e     2.000000e-01
EX_glu_L_e     1.589140e-01
EX_ile_L_e     2.000000e-01
EX_leu_L_e     2.000000e-01
EX_nac_e       1.200000e-03
EX_nh4_e       5.038239e+00
EX_phe_L_e     2.000000e-01
EX_pi_e        7.524606e-01
EX_pnto_R_e    1.200000e-04
EX_pydam_e     6.000000e-07
EX_val_L_e     2.000000e-01
dtype: float64


/home/bras5181/miniconda3/lib/python3.8/site-packages/cobra/medium/minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()
